In [1]:
import pandas as pd
import numpy as np
import pprint
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from datetime import date

In [15]:
class IFSCScraper():
    """
    Define a class for the scraper that will be used to gather data from the IFSC website
    (ifsc-climbing.org)
    Includes methods that allow for scraping different pages and different information
    """
    # Page url
    url = 'https://www.ifsc-climbing.org/index.php/world-competition/last-result'
    
    def __init__(self, debug=False):
        """
        Initialize a scraper object with its own browser instance
        Input:
            debug - Indicates whether this is a debug instance for quicker development
        """
        self.debug = debug
        self.year_events = {}

        self.generate_driver()
        time.sleep(1)
    
    def generate_driver(self):
        """
        Initialize Selenium web browser
        Input:
            N/A
        """
        try:
            self.driver = webdriver.Firefox()
        except:
            print('Error: Could not create WebDriver object...')
        
    def load_page(self, link, timeout=10, wait_after=5):
        """
        Helper function that loads a page and waits for timeout
        input:
            link - Link to the page we wish to load
            timeout - Seconds to wait before timing out
            wait_after - Seconds to wait after loading
        output:
            N/A
        """

        # Visit link
        self.driver.get(link)

        # Attempt to open link
        try:
            WebDriverWait(self.driver, timeout).until(EC.visibility_of_element_located((By.XPATH,
            "//div[@class='uk-container']")))
        except TimeoutException:
            print("Timed out waiting for page " + link + " to load")
            self.driver.quit()

        # Wait for page to load
        time.sleep(wait_after)
        
    def get_comp_years_and_league(self):
        """
        Parse the world-competition/last-result page to find and return years and leagues
        input:
            N/A
        output:
            List of touples containing comp year and league
        """
        year_league_comb = []
        
        try:
            self.load_page(IFSCScraper.url)
        except:
            print('Error loading page!')
            self.driver.quit()
            
        # The dropdown menus to pick years/leagues/events is in an iframe - we need to switch to it
        frame = self.driver.find_element(By.XPATH, '/html/body/div[1]/div[4]/div/iframe')
        self.driver.switch_to.frame(frame)

        # Dropdown menus for each choice
        year_dd     = self.driver.find_element(By.XPATH, '//select[@id="years"]')
        league_dd   = self.driver.find_element(By.XPATH, '//select[@id="indexes"]')

        # Select all options for 'Year' and 'League' dropdown menus
        year_opts = Select(year_dd).options
        league_opts = Select(league_dd).options

        # Extract text of each of the above options
        years   = [opt.text for opt in year_opts]
        # leagues = [opt.text for opt in league_opts[1:]] # all leagues
        league = league_opts[1].text #world cup only for now

        year_league_comb = []
        for year in years:
            if (year, league) not in year_league_comb:
                year_league_comb.append((year, league))

        return year_league_comb

    def get_years_events(self, years):
        """
        Iterate through each year and get that years events
        input:
            N/A
        output:
            List of tuples containing comp year and league
        """
        try:
            self.load_page(IFSCScraper.url)
        except:
            print('Error loading page!')
        
        # The dropdown menus to pick years/leagues/events is in an iframe - we need to switch to it
        frame = self.driver.find_element(By.XPATH, '/html/body/div[1]/div[4]/div/iframe')
        self.driver.switch_to.frame(frame)

        # Dropdown menus for each choice
        year_dd, league_dd, event_dd, cat_dd = self.get_dropdowns(self.driver)
                
        # Creating wait
        wait = WebDriverWait(self.driver, 10)
        
        # Iterate through years provided by get_comp_years_and_league
        for year in years:                
            years_ob   = Select(year_dd).select_by_value(year[0])   #0 is most recent year (2023)
            leagues_ob = Select(league_dd).select_by_index(1) #starts at index 1
        
            # IF THINGS BREAK, UNCOMMENT THIS SECTION
            # Selects third dropdown menu, events
            # event_dd = self.driver.find_element(By.XPATH, '//select[@id="events"]')
            
            # Waits for third dropdown to populate with options based on leagues_ob                                        
            events_select = Select(event_dd)
            wait.until(lambda d: len(events_select.options) > 1)
            
            # Gets event options and ids for year and add to dictionary
            event_opts = Select(event_dd).options
            events = [opt.text for opt in event_opts[1:]]
            event_ids = [opt.get_attribute("value").split('/')[-1] for opt in event_opts[1:]]
            self.year_events[year[0]] = [(event, id) for event, id in zip(events, event_ids)]
            
            print(f'{year[0]}: {self.check_for_event_results(self.driver, event_dd, events)}')
    
    def check_for_event_results(self, driver, dd, events):
        """
        Iterate through each event and check if results exist
        input:
            N/A
        output:
            unsure yet
        """
        event_has_results = []
        for event in events:
            # Select event in events dropdown
            events_ob = Select(dd).select_by_visible_text(event)

            # Select fourth dropdown menu, categories
            cat_dd = driver.find_element(By.XPATH, '//select[@id="categories"]')
            cat_select = Select(cat_dd)
            wait = WebDriverWait(driver, 2)
            try:
                wait.until(lambda d: len(cat_select.options) > 1)
                event_has_results.append(1)
            except:
                event_has_results.append(0)
        return f'{sum(event_has_results)} of {len(events)} events have results!'
    
    def get_single_year(self, year = '2022'):
        """
        Iterate through each event and check if results exist
        input:
            driver - selenium instance
            year (string) - year to be scraped
        output:
            df of that year's results
        """
        try:
            self.load_page(IFSCScraper.url)            
            wait = WebDriverWait(self.driver, 10)
        except:
            print('Error loading page!')
        
        wait.until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe.jch-lazyloaded")))
        
        # Dropdown menus for each choice
        year_dd, league_dd, event_dd, cat_dd = self.get_dropdowns(self.driver)
        
        # Select given year and league
        year_ob   = Select(year_dd).select_by_visible_text(year)
        league_ob = Select(league_dd).select_by_index(1)
        
        #need to loop through events here!
        all_events = self.get_events(self.driver, event_dd)
        
        ######## TODO: implement another loop for *every* event!        
        
        category_select = Select(cat_dd)
        wait.until(lambda d: len(category_select.options) > 1)
        
        dfs = []
        for cat in category_select.options[1:]:
            cat_ob = Select(cat_dd).select_by_visible_text(cat.text) # selects category
            
            # Finds table with desired data 
            wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@id="table_id_wrapper"]')))
            table_wrapper = self.driver.find_element(By.XPATH, '//div[@id="table_id_wrapper"]')
            results = table_wrapper.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')
            
            # Get event name and date
            event_details = self.driver.find_element(By.XPATH, '//div[@class="labels"]')
            event_results = event_details.find_elements(By.TAG_NAME, 'p')
                        
            # Gets data and stores it in dictionary
            data = []
            for result in results:                
                details = result.find_elements(By.TAG_NAME, 'td')
                temp_dict = {
                    "Rank": details[0].text,
                    "Name": f"{details[1].text} {details[2].text}",
                    "Country": details[3].text,
                    "Qualification": details[4].text,
                    "Semi-Final": details[5].text,
                    "Final": details[6].text
                }
                data.append(temp_dict)
                
            # Create dataframe after collecting all the data
            df = pd.DataFrame.from_dict(data)
            dfs.append((cat.text, event_results[0].text, event_results[1].text, df))
        return dfs            

    def convert_to_csv(self, packed_data):
        (category, event, date, data) = packed_data
            
        return category, event, date, data
    
    def get_dropdowns(self, driver):
        year_dd   = driver.find_element(By.XPATH, '//select[@id="years"]')
        league_dd = driver.find_element(By.XPATH, '//select[@id="indexes"]')
        event_dd  = driver.find_element(By.XPATH, '//select[@id="events"]')
        cat_dd    = driver.find_element(By.XPATH, '//select[@id="categories"]')
        
        return year_dd, league_dd, event_dd, cat_dd
    
    def get_events(self, driver, events_dd):
        event_opts = Select(events_dd)
        wait = WebDriverWait(driver, 10)
        wait.until(lambda d: len(event_opts.options) > 1)
                    
        return [x.text for x in event_opts.options[1:]]
        
    def end_session(self):
        self.driver.quit()
    
    def scrape_site(self):
        self.get_years_events(self.get_comp_years_and_league())
        self.end_session()
        
def display_events(dict):
        for k in dict:
            print(f'{int(k)}:')
            for v in dict[k]:
                print('    ', v)

In [16]:
scraper = IFSCScraper()
temp = scraper.get_single_year()
scraper.end_session()
# scraper.scrape_site()

all events: ['IFSC - Climbing World Cup (B) - Meiringen (SUI) 2022', 'IFSC - Climbing World Cup (B,S) - Seoul (KOR) 2022', 'IFSC - Climbing World Cup (B,S) - Salt Lake City (USA) 2022', 'IFSC - Climbing World Cup (B,S) - Salt Lake City (USA) 2022', 'IFSC - Climbing World Cup (B) - Brixen (ITA) 2022', 'IFSC - Climbing World Cup (B,L) - Innsbruck (AUT) 2022', 'IFSC - Climbing World Cup (L,S) - Villars (SUI) 2022', 'IFSC - Climbing World Cup (L,S) - Chamonix (FRA) 2022', 'IFSC - Climbing World Cup (L) - Briançon (FRA) 2022', 'IFSC - Climbing World Cup (L) - Koper (SLO) 2022', 'IFSC - Climbing World Cup (L,S) - Edinburgh (GBR) 2022', 'IFSC - Climbing World Cup (L,S) - Jakarta (INA) 2022', 'IFSC - Climbing World Cup (B&L) - Morioka, Iwate (JPN) 2022']


In [ ]:
for item in temp:
    scraper.convert_to_csv(item)

In [ ]:
# display_events(scraper.year_events)